# American Option Pricing using Binomial Tree (CRR Model)

In this notebook, we implement the Cox-Ross-Rubinstein binomial tree model to price American options.  
Unlike European options, American options can be exercised at any time before expiration.  
This feature requires backward induction through the binomial tree to check for early exercise opportunities.


In [6]:
# User input for American option parameters
try:
    S = float(input("Enter the current stock price (S): "))
    K = float(input("Enter the strike price (K): "))
    T = float(input("Enter the time to maturity (in years, T): "))
    r = float(input("Enter the annual risk-free interest rate (r): "))
    sigma = float(input("Enter the annual volatility (σ): "))
    N = int(input("Enter the number of time steps (N): "))
    option_type = input("Enter the option type ('call' or 'put'): ").lower().strip()

    assert option_type in {"call", "put"}, "Invalid option type."

except Exception as e:
    print(f"Error: {e}")

Error: could not convert string to float: ''


Price an American option using the Cox-Ross-Rubinstein binomial model.

Parameters
---
S: Current stock price

K: Strike price

T: Time to maturity (in years)

r: Annual risk-free interest rate

sigma: Annual volatility

N: Number of time steps

option_type: Either 'call' or 'put'

Returns:     American option price

In [5]:
import numpy as np

def binomial_american(
    S: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    N: int,
    option_type: str = "call"
) -> float:


    if option_type not in {"call", "put"}:
        raise ValueError("option_type must be 'call' or 'put'")

    dt = T / N
    discount = np.exp(-r * dt)
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Stock prices at maturity
    ST = np.array([S * (u ** j) * (d ** (N - j)) for j in range(N + 1)])

    # Option values at maturity
    if option_type == "call":
        values = np.maximum(0, ST - K)
    else:
        values = np.maximum(0, K - ST)

    # Backward induction
    for i in range(N - 1, -1, -1):
        ST = ST[:i + 1] / u  # update prices one step back
        values = discount * (p * values[1:i + 2] + (1 - p) * values[0:i + 1])
        exercise = (ST - K) if option_type == "call" else (K - ST)
        values = np.maximum(values, exercise)

    return values[0]


#### Difference between Black Sholes and Binomial Tree

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

def black_scholes(
    S: float, K: float, T: float, r: float, sigma: float, option_type: str = "call"
) -> float:
    
    """
    Black-Scholes price for European options (used to compare).
    """

    if option_type not in {"call", "put"}:
        raise ValueError("option_type must be 'call' or 'put'")

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


### With my own data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm



# -------------------------------
# Black-Scholes function (European)
# -------------------------------

def black_scholes(S, K, T, r, sigma, option_type='call'):

    if option_type not in {'call', 'put'}:
        raise ValueError("option_type must be 'call' or 'put'")
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)



# -------------------------------
# Binomial American function
# -------------------------------

def binomial_american(S, K, T, r, sigma, N, option_type='call'):

    if option_type not in {'call', 'put'}:
        raise ValueError("option_type must be 'call' or 'put'")
    
    dt = T / N
    discount = np.exp(-r * dt)
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    ST = np.array([S * (u ** j) * (d ** (N - j)) for j in range(N + 1)])

    if option_type == 'call':
        values = np.maximum(0, ST - K)
    else:
        values = np.maximum(0, K - ST)

    for i in range(N - 1, -1, -1):
        ST = ST[:i + 1] / u
        values = discount * (p * values[1:i + 2] + (1 - p) * values[0:i + 1])
        exercise = (ST - K) if option_type == "call" else (K - ST)
        values = np.maximum(values, exercise)

    return values[0]



# -------------------------------
# Option Data (Tempus AI)
# -------------------------------

options_data = [
    {'option_type': 'call', 'K': 30, 'market_price': 29.80, 'imp_vol': 0.023, 'T': 0.27},
    {'option_type': 'call', 'K': 35, 'market_price': 24.80, 'imp_vol': 0.018, 'T': 0.27},
    {'option_type': 'call', 'K': 40, 'market_price': 20.10, 'imp_vol': 0.017, 'T': 0.27},
    {'option_type': 'put',  'K': 30, 'market_price': 0.03,  'imp_vol': 0.019, 'T': 0.27},
    {'option_type': 'put',  'K': 40, 'market_price': 0.05,  'imp_vol': 0.012, 'T': 0.27},
]



# -------------------------------
# Common parameters
# -------------------------------

S = 37    # Current stock price
r = 0.05  # Risk-free interest rate
N = 100   # Time steps for binomial model



# -------------------------------
# Calculate and store results
# -------------------------------

results = []

for option in options_data:
    K_i = option['K']
    T_i = option['T']
    sigma_i = option['imp_vol']
    opt_type = option['option_type']
    market = option['market_price']

    bs_price = black_scholes(S, K_i, T_i, r, sigma_i, opt_type)
    bin_price = binomial_american(S, K_i, T_i, r, sigma_i, N, opt_type)

    results.append({
        "Option Type": opt_type.upper(),
        "Strike (K)": K_i,
        "T (years)": T_i,
        "Market Price": round(market, 2),
        "BS Price": round(bs_price, 2),
        "Binomial Price": round(bin_price, 2),
        "BS Abs Error": round(abs(bs_price - market), 4),
        "Binomial Abs Error": round(abs(bin_price - market), 4)
    })


# -------------------------------
# Create DataFrame and export
# -------------------------------

df = pd.DataFrame(results)
display(df)

# Save to CSV
df.to_csv("../data/tempus_options_comparison.csv", index=False)


,Option Type,Strike (K),T (years),Market Price,BS Price,Binomial Price,BS Abs Error,Binomial Abs Error
0,CALL,30,0.27,29.80,7.40,7.40,22.3977,22.3977
1,CALL,35,0.27,24.80,2.47,2.47,22.3307,22.3307
2,CALL,40,0.27,20.10,0.00,0.00,20.1000,20.1000
3,PUT,30,0.27,0.03,0.00,0.00,0.0300,0.0300
4,PUT,40,0.27,0.05,2.46,7.34,2.4136,7.2881
